In [ ]:
import sys
sys.path.append("../Modules/")
sys.path.append("../pyFiles/")
sys.path.append("../")
import os

import pandas as pd
import numpy as np

# from configobj import ConfigObj
# from configobj import flatten_errors
# from validate import Validator
    
import logging

# from analysisParameters import loadParameters
# from loadDataE1 import loadSessionDict

#fmt = 'logging.Formatter(''%(levelname)s_%(name)s-%(funcName)s(): - %(message)s'
fmt = '%(levelname)s_%(name)s-%(funcName)s(): - %(message)s'
logging.basicConfig(level=logging.INFO, format=fmt)
logger = logging.getLogger(__name__)

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import os

init_notebook_mode(connected=True)


In [2]:
# sessionDict = projectTrajectories(sessionDict)
import matplotlib.pyplot as plt
%matplotlib notebook

In [3]:
import pickle

file = open('../sessionFiles.pickle', 'rb')
allSessions = pickle.load(file)
file.close()

sessionDict = allSessions[0]

In [4]:
list(sessionDict.keys())

['subID',
 'trialInfo',
 'expConfig',
 'rawExpUnity',
 'rawExpGaze',
 'processedExp',
 'rawCalibUnity',
 'rawCalibGaze',
 'processedCalib',
 'analysisParameters']

In [5]:
processedCalib = sessionDict['processedCalib']
list(processedCalib)

[('azimuthWidth', ''),
 ('base_data', ''),
 ('blockNumber', ''),
 ('camera', 'R0C0'),
 ('camera', 'R0C1'),
 ('camera', 'R0C2'),
 ('camera', 'R0C3'),
 ('camera', 'R1C0'),
 ('camera', 'R1C1'),
 ('camera', 'R1C2'),
 ('camera', 'R1C3'),
 ('camera', 'R2C0'),
 ('camera', 'R2C1'),
 ('camera', 'R2C2'),
 ('camera', 'R2C3'),
 ('camera', 'R3C0'),
 ('camera', 'R3C1'),
 ('camera', 'R3C2'),
 ('camera', 'R3C3'),
 ('cameraPos', 'x'),
 ('cameraPos', 'y'),
 ('cameraPos', 'z'),
 ('cameraRot', 'x'),
 ('cameraRot', 'y'),
 ('cameraRot', 'z'),
 ('confidence', ''),
 ('currentTargetName', ''),
 ('elevationHeight', ''),
 ('eye-center0-3d', 'x'),
 ('eye-center0-3d', 'y'),
 ('eye-center0-3d', 'z'),
 ('eye-center1-3d', 'x'),
 ('eye-center1-3d', 'y'),
 ('eye-center1-3d', 'z'),
 ('fixationTime', ''),
 ('frameTime', ''),
 ('gaze-normal0', 'x'),
 ('gaze-normal0', 'y'),
 ('gaze-normal0', 'z'),
 ('gaze-normal1', 'x'),
 ('gaze-normal1', 'y'),
 ('gaze-normal1', 'z'),
 ('gaze-point-3d', 'x'),
 ('gaze-point-3d', 'y'),
 ('ga

In [6]:
processedCalib['worldVidTimeSec'] = np.array(processedCalib['pupilTimestamp'] - processedCalib['pupilTimestamp'].iloc[0])
processedCalib['worldVidTimeSec'].tail(10)

2965    26.072226
2966    26.076210
2967    26.076532
2968    26.080378
2969    26.084560
2970    26.087518
2971    26.092663
2972    26.096482
2973    26.098138
2974    26.102518
Name: worldVidTimeSec, dtype: float64

In [7]:
list(processedCalib)

[('azimuthWidth', ''),
 ('base_data', ''),
 ('blockNumber', ''),
 ('camera', 'R0C0'),
 ('camera', 'R0C1'),
 ('camera', 'R0C2'),
 ('camera', 'R0C3'),
 ('camera', 'R1C0'),
 ('camera', 'R1C1'),
 ('camera', 'R1C2'),
 ('camera', 'R1C3'),
 ('camera', 'R2C0'),
 ('camera', 'R2C1'),
 ('camera', 'R2C2'),
 ('camera', 'R2C3'),
 ('camera', 'R3C0'),
 ('camera', 'R3C1'),
 ('camera', 'R3C2'),
 ('camera', 'R3C3'),
 ('cameraPos', 'x'),
 ('cameraPos', 'y'),
 ('cameraPos', 'z'),
 ('cameraRot', 'x'),
 ('cameraRot', 'y'),
 ('cameraRot', 'z'),
 ('confidence', ''),
 ('currentTargetName', ''),
 ('elevationHeight', ''),
 ('eye-center0-3d', 'x'),
 ('eye-center0-3d', 'y'),
 ('eye-center0-3d', 'z'),
 ('eye-center1-3d', 'x'),
 ('eye-center1-3d', 'y'),
 ('eye-center1-3d', 'z'),
 ('fixationTime', ''),
 ('frameTime', ''),
 ('gaze-normal0', 'x'),
 ('gaze-normal0', 'y'),
 ('gaze-normal0', 'z'),
 ('gaze-normal1', 'x'),
 ('gaze-normal1', 'y'),
 ('gaze-normal1', 'z'),
 ('gaze-point-3d', 'x'),
 ('gaze-point-3d', 'y'),
 ('ga

### Calc cycEIH_az / cycEIH_el

In [19]:
df2 = processedCalib

def calcGazeAzEl(row):
    
    x = row[('gaze-normal0','x')] 
    y = row[('gaze-normal0','y')] 
    z = row[('gaze-normal0','z')] 
    
    az = np.rad2deg(np.arctan(np.divide(x,z)))
    el = np.rad2deg(np.arctan(np.divide(y,z)))
    
    x = row[('gaze-normal1','x')] 
    y = row[('gaze-normal1','y')] 
    z = row[('gaze-normal1','z')] 
    
    az2 = np.rad2deg(np.arctan(np.divide(x,z)))
    el2 = np.rad2deg(np.arctan(np.divide(y,z)))
    
    #Note that the .apply requires you return a single data structure, 
    #so a SINGLE tuple is OK, but the seperate values for az/el are not OK.
    
    return (np.nanmean([az,az2]),np.nanmean([el,el2])) 


azEl = df2.apply(lambda arbitraryRowName: calcGazeAzEl(arbitraryRowName),axis=1)


# df2[('gazeInWorldCam','az')],df2[('gazeInWorldCam','el')]= zip(*azEl)
df2[('cycEIH_az')],df2[('cycEIH_el')]= zip(*azEl)
df2['deltaT'] = df2['worldVidTimeSec'].diff()


/var/folders/gy/wzd471c14k762xp4b_xt87xr0000gq/T/ipykernel_78422/2669729467.py:22: RuntimeWarning:

Mean of empty slice



In [20]:

def calcTargetAzEl(row):
    
    x = row['targeLocalPos','x']
    y = row['targeLocalPos','y']
    z = row['targeLocalPos','z']
    
    az = np.rad2deg(np.arctan(np.divide(x,z)))
    el = np.rad2deg(np.arctan(np.divide(y,z)))
    
    #Note that the .apply requires you return a single data structure, 
    #so a SINGLE tuple is OK, but the seperate values for az/el are not OK.
    
    return (az,el) 

ballAzEl = df2.apply(lambda arbitraryRowName: calcTargetAzEl(arbitraryRowName),axis=1)

df2['ball_az'],df2['ball_el'] = zip(*ballAzEl)
df2['ballVelInWorldCam_az'] = df2['ball_az'].diff() / df2['deltaT']
df2['ballVelInWorldCam_el'] = df2['ball_el'].diff() / df2['deltaT']
df2['ballVelInWorldCam'] = np.abs(np.sqrt(df2['ballVelInWorldCam_az'].diff()**2  + df2['ballVelInWorldCam_el'].diff()**2) / df2['deltaT'])


## Plot calib assessment using local spherical (EIH)

In [21]:
# yLim = [-100,100]

yLim=[-50,50]
width=800
height=600



startTimeS = df2['worldVidTimeSec'].iloc[0]
endTimeS = df2['worldVidTimeSec'].iloc[-2]

# I googled for, "python find first larger than"
startIndex = next(idx for idx, val in enumerate(df2['worldVidTimeSec']) if val >  startTimeS)
endIndex = next(idx for idx, val in enumerate(df2['worldVidTimeSec']) if val >  endTimeS)
print('Plotting from index: ' + str(startIndex) + ' to ' + str(endIndex))

colors_idx = ['rgb(0,204,204)','rgb(128,128,128)','rgb(204,0,0)','rgb(102,0,204)']

# from plotly.subplots import make_subplots
# fig = make_subplots(rows=2, cols=1,shared_xaxes=True)

traces = []

###

bEih_el = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=df2['ball_el'][startIndex:endIndex],
    name = 'ball_el',
    marker_color = colors_idx[2],
    mode='markers',
    marker_size = 8,
    opacity = 0.5)

bEih_az = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=df2['ball_az'][startIndex:endIndex],
    name = 'ball_az',
    marker_color = colors_idx[3],
    mode='markers',
    marker_size = 8,
    opacity = 0.5)


traces.append(bEih_el)
traces.append(bEih_az)


eih_el = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=df2['cycEIH_el'][startIndex:endIndex],
    name = 'cycEIH_el',
    marker_color = colors_idx[0],
    mode='markers',
    marker_size = 5,
    opacity = 1)

eih_az = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=df2['cycEIH_az'][startIndex:endIndex],
    name = 'cycEIH_az',
    marker_color = colors_idx[1],
    mode='markers',
    marker_size = 5,
    opacity = 1)

traces.append(eih_el)
traces.append(eih_az)

layout = dict(
    dragmode= 'pan',
    title='Time Series with Rangeslider',
    width=width,
    height=height,
    yaxis=dict(range=yLim, title='angular position (degrees)'),
    xaxis=dict(
        rangeslider=dict(visible=True))
)


fig = dict(data=traces, layout=layout)

iplot(fig)


Plotting from index: 1 to 2974


In [22]:
def calcGazeAzEl(row):
    
    x = row[('gaze-point-3d','x')] 
    y = row[('gaze-point-3d','y')]
    z = row[('gaze-point-3d','z')]
    
#     x = row[('gaze-point-3d','x')] - row[('cameraPos','x')]
#     y = row[('gaze-point-3d','y')] - row[('cameraPos','y')]
#     z = row[('gaze-point-3d','z')] - row[('cameraPos','z')]
    
    az = np.rad2deg(np.arctan(np.divide(x,z)))
    el = np.rad2deg(np.arctan(np.divide(y,z)))
    
    #Note that the .apply requires you return a single data structure, 
    #so a SINGLE tuple is OK, but the seperate values for az/el are not OK.
    
    return (az,el) 


azEl = df2.apply(lambda arbitraryRowName: calcGazeAzEl(arbitraryRowName),axis=1)

df2['fromGazePoint3d_az'],df2['fromGazePoint3d_el'] = zip(*azEl)



### This figure shows that gaze-point-3d data is in head centered coordinates

In [23]:
# yLim = [-100,100]

yLim=[-50,50]
width=800
height=600

startTimeS = df2['worldVidTimeSec'].iloc[0]
endTimeS = df2['worldVidTimeSec'].iloc[-2]

# I googled for, "python find first larger than"
startIndex = next(idx for idx, val in enumerate(df2['worldVidTimeSec']) if val >  startTimeS)
endIndex = next(idx for idx, val in enumerate(df2['worldVidTimeSec']) if val >  endTimeS)
print('Plotting from index: ' + str(startIndex) + ' to ' + str(endIndex))

colors_idx = ['rgb(0,204,204)','rgb(128,128,128)','rgb(204,0,0)','rgb(102,0,204)', 'rgb(150,50,50)', 'rgb(50,50,150)']

# from plotly.subplots import make_subplots
# fig = make_subplots(rows=2, cols=1,shared_xaxes=True)

traces = []

###

bEih_el = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=df2['ball_el'][startIndex:endIndex],
    name = 'ball_el',
    marker_color = colors_idx[2],
    mode='markers',
    marker_size = 8,
    opacity = 0.5)

bEih_az = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=df2['ball_az'][startIndex:endIndex],
    name = 'ball_az',
    marker_color = colors_idx[3],
    mode='markers',
    marker_size = 8,
    opacity = 0.5)

traces.append(bEih_el)
traces.append(bEih_az)

eih_el = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=df2['cycEIH_el'][startIndex:endIndex],
    name = 'cycEIH_el',
    marker_color = colors_idx[0],
    mode='markers',
    marker_size = 8,
    opacity = 1)

eih_az = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=df2['cycEIH_az'][startIndex:endIndex],
    name = 'cycEIH_az',
    marker_color = colors_idx[1],
    mode='markers',
    marker_size = 8,
    opacity = 1)

traces.append(eih_el)
traces.append(eih_az)


#####


fromGP3D_el = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=df2['fromGazePoint3d_el'][startIndex:endIndex],
    name = 'fromGazePoint3d_el',
    marker_color = colors_idx[4],
    mode='markers',
    marker_size = 3,
    opacity = .5)

fromGP3D_az = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=df2['fromGazePoint3d_az'][startIndex:endIndex],
    name = 'fromGazePoint3d_az',
    marker_color = colors_idx[5],
    mode='markers',
    marker_size = 3,
    opacity = .5)

traces.append(fromGP3D_el)
traces.append(fromGP3D_az)


layout = dict(
    dragmode= 'pan',
    title='Time Series with Rangeslider',
    width=width,
    height=height,
    yaxis=dict(range=yLim, title='angular position (degrees)'),
    xaxis=dict(
        rangeslider=dict(visible=True))
)


fig = dict(data=traces, layout=layout)

iplot(fig)



Plotting from index: 1 to 2974


In [13]:
# # yLim = [-100,100]

# yLim=[-50,50]
# width=800
# height=600

# startTimeS = df2['worldVidTimeSec'].iloc[0]
# endTimeS = df2['worldVidTimeSec'].iloc[-2]

# # I googled for, "python find first larger than"
# startIndex = next(idx for idx, val in enumerate(df2['worldVidTimeSec']) if val >  startTimeS)
# endIndex = next(idx for idx, val in enumerate(df2['worldVidTimeSec']) if val >  endTimeS)
# print('Plotting from index: ' + str(startIndex) + ' to ' + str(endIndex))

# colors_idx = ['rgb(0,204,204)','rgb(128,128,128)','rgb(204,0,0)','rgb(102,0,204)', 'rgb(150,50,50)', 'rgb(50,50,150)']

# # from plotly.subplots import make_subplots
# # fig = make_subplots(rows=2, cols=1,shared_xaxes=True)

# traces = []

# ###

# bEih_el = go.Scattergl(
#     x=df2['worldVidTimeSec'][startIndex:endIndex],
#     y=df2['ball_el'][startIndex:endIndex],
#     name = 'ball_el',
#     marker_color = colors_idx[2],
#     mode='markers',
#     marker_size = 8,
#     opacity = 0.5)

# bEih_az = go.Scattergl(
#     x=df2['worldVidTimeSec'][startIndex:endIndex],
#     y=df2['ball_az'][startIndex:endIndex],
#     name = 'ball_az',
#     marker_color = colors_idx[3],
#     mode='markers',
#     marker_size = 8,
#     opacity = 0.5)


# eih_el = go.Scattergl(
#     x=df2['worldVidTimeSec'][startIndex:endIndex],
#     y=-df2['gazeInWorldCam_el'][startIndex:endIndex],
#     name = 'gazeInWorldCam_el',
#     marker_color = colors_idx[0],
#     mode='markers',
#     marker_size = 8,
#     opacity = 1)

# eih_az = go.Scattergl(
#     x=df2['worldVidTimeSec'][startIndex:endIndex],
#     y=df2['gazeInWorldCam_az'][startIndex:endIndex],
#     name = 'gazeInWorldCam_az',
#     marker_color = colors_idx[1],
#     mode='markers',
#     marker_size = 8,
#     opacity = 1)

# traces.append(eih_el)
# traces.append(eih_az)


# #####


# fromGP3D_el = go.Scattergl(
#     x=df2['worldVidTimeSec'][startIndex:endIndex],
#     y=-df2['fromGazePoint3d_el'][startIndex:endIndex],
#     name = 'fromGazePoint3d_el',
#     marker_color = colors_idx[4],
#     mode='markers',
#     marker_size = 3,
#     opacity = .5)

# fromGP3D_az = go.Scattergl(
#     x=df2['worldVidTimeSec'][startIndex:endIndex],
#     y=df2['fromGazePoint3d_az'][startIndex:endIndex],
#     name = 'fromGazePoint3d_az',
#     marker_color = colors_idx[5],
#     mode='markers',
#     marker_size = 3,
#     opacity = .5)

# traces.append(fromGP3D_el)
# traces.append(fromGP3D_az)


# layout = dict(
#     dragmode= 'pan',
#     title='Time Series with Rangeslider',
#     width=width,
#     height=height,
#     yaxis=dict(range=yLim, title='angular position (degrees)'),
#     xaxis=dict(
#         rangeslider=dict(visible=True))
# )


# fig = dict(data=traces, layout=layout)

# iplot(fig)


